In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
import random
import torch
from typing import List, Dict, Optional
import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna
from optuna.pruners import MedianPruner
from optuna.exceptions import TrialPruned
import warnings

In [11]:
# for tuning
CONFIG = {
    'num_partitions': 3, # max 34
    'tuning_epoch': 12,
    'checkpoint_interval': 5,
    'eval_interval': 3,
    'num_tuning_trials': 30,
    'seed': 42 # or None
}

In [12]:
sys.path.append('..') 

from src.models.autoencoder import AutoEncoder
from src.models.autoencoder_trainer import *
from src.data.data_utils import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
data_dir = '../NETFLIX_DATA/partitions/train'
val_dir = '../NETFLIX_DATA/partitions/validation'
checkpoint_dir = '../model_checkpoints'

In [5]:
# retreive training data info
train_partition_files = get_data(data_dir)
print(f"Number of training partitions: {len(train_partition_files)}")
val_partition_files = get_data(val_dir)
print(f"Number of validation partitions: {len(val_partition_files)}")

if CONFIG['seed'] is not None:
    random.seed(CONFIG['seed'])

# testing
sample_train_partitions = random.sample(train_partition_files, CONFIG['num_partitions'])

sample_val_partitions = []
for partition in sample_train_partitions:
  val_partition = partition.copy()
  val_partition['path'] = partition['path'].replace('train', 'validation')
  sample_val_partitions.append(val_partition)


print(f"Train EX: {sample_train_partitions[0]}")
print(f"Val EX: {sample_val_partitions[0]}")

Number of training partitions: 34
Number of validation partitions: 34
Train EX: {'path': '../NETFLIX_DATA/partitions/train/part_1_7.parquet', 'part': 1, 'group': 7}
Val EX: {'path': '../NETFLIX_DATA/partitions/validation/part_1_7.parquet', 'part': 1, 'group': 7}


In [6]:
# build user_map and movie_map
user_map, movie_map = map_id(sample_train_partitions)

Mapping IDs: 100%|██████████| 3/3 [00:00<00:00,  3.61it/s]

Map successful for 406244 users, 1653 movies


In [7]:
# preload user rating profiles

train_user_data = AutoEncoder.load_user_data(partitions=sample_train_partitions, 
                                             user_map=user_map)

validation_user_data = AutoEncoder.load_validation_data(partitions=sample_val_partitions, 
                                                        user_map=user_map,
                                                        movie_map=movie_map)

Loading validation pairs 3/3: 100%|██████████| 338140/338140 [00:10<00:00, 31777.64it/s]


In [8]:
# optuna objective
def objective(trial):
    params = {
        "num_epochs": CONFIG['tuning_epoch'],
        "batch_size": 512,
        "learning_rate": trial.suggest_float("learning_rate", 0.0001, 0.001, log=True),
        "hidden_dims": trial.suggest_categorical("hidden_dims", 
                                                 [[1024,256,128], [512,256,128], [256,128], [512,128]]),
        "dropout": trial.suggest_float("dropout", 0.3, 0.7),
        "l2_reg": trial.suggest_float("l2_reg", 0.00001, 0.01, log=True),
        "checkpoint_interval": CONFIG['checkpoint_interval'],
        "eval_interval": CONFIG['eval_interval'],
    }

                #[[512,256,128], [256,128], [512,128]]

    try:
        print(params)
        
        model, rmse = train_autoencoder(
            train_partitions=sample_train_partitions,
            user_map=user_map,
            movie_map=movie_map,
            validation_partitions=sample_val_partitions,
            checkpoint_dir=checkpoint_dir,
            trial=trial,
            user_data=train_user_data,
            validation_data=validation_user_data,
            **params
        )

        return float(rmse)
    except optuna.TrialPruned:
        raise # reraise prune error for tuning

In [ ]:
# tuning
warnings.filterwarnings("ignore", module="optuna.*")

study = optuna.create_study(
    study_name=f"autoencoder_tuning_{CONFIG['num_partitions']}_samples", 
    direction='minimize',
    
    # prune after 2 trials, after 1 if really bad
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=6, interval_steps=2),
    
    sampler=optuna.samplers.TPESampler(),
    storage=f"sqlite:///optuna_study_{CONFIG['num_partitions']}_samples.db",
    load_if_exists=True
)

study.optimize(objective, n_trials=CONFIG['num_tuning_trials'], timeout=6*3600)

[I 2025-06-13 13:00:24,005] Using an existing study with name 'autoencoder_tuning_3_samples' instead of creating a new one.


{'num_epochs': 12, 'batch_size': 512, 'learning_rate': 0.0002172479460451583, 'hidden_dims': [1024, 256, 128], 'dropout': 0.37158451497450784, 'l2_reg': 2.7264522069379994e-05, 'checkpoint_interval': 5, 'eval_interval': 3}
Device: cuda


Calculating global mean: 100%|██████████| 3/3 [00:00<00:00,  7.84it/s]


Global mean rating: 3.582


Epoch 1/12: 100%|██████████| 794/794 [01:19<00:00,  9.96it/s, loss=1.0174] 


...

Epoch 4/12: 100%|██████████| 794/794 [01:18<00:00, 10.06it/s, loss=0.9729]


Epoch 4 - Average Loss: 0.9586


Epoch 5/12: 100%|██████████| 794/794 [01:18<00:00, 10.15it/s, loss=0.9242]


Epoch 5 - Average Loss: 0.9245
Saved checkpoint at ../model_checkpoints/checkpoint_epoch_5.pth.gz


Epoch 6/12:  32%|███▏      | 255/794 [00:25<00:53, 10.12it/s, loss=0.9218]

In [9]:
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

study = optuna.load_study(
    storage="sqlite:///optuna_study_3_samples.db", 
    study_name="autoencoder_tuning_3_samples"       
)

print(f"Best RMSE: {study.best_value:.4f}")
print(f"Best params: {study.best_params}")
print(f"Total trials: {len(study.trials)}")

Best RMSE: 0.9725
Best params: {'learning_rate': 0.00025806243863933984, 'hidden_dims': [1024, 256, 128], 'dropout': 0.4807743363021898, 'l2_reg': 3.332750186337578e-05}
Total trials: 46


In [14]:
# retreive full training data info
train_partition_files = get_data(data_dir)
val_partition_files = get_data(val_dir)

if CONFIG['seed'] is not None:
    random.seed(CONFIG['seed'])

full_train_partitions = random.sample(train_partition_files, 7)

full_val_partitions = []
for partition in full_train_partitions:
  val_partition = partition.copy()
  val_partition['path'] = partition['path'].replace('train', 'validation')
  full_val_partitions.append(val_partition)


print(f"Train EX: {sample_train_partitions[0]}")
print(f"Val EX: {sample_val_partitions[0]}")

Train EX: {'path': '../NETFLIX_DATA/partitions/train/part_1_7.parquet', 'part': 1, 'group': 7}
Val EX: {'path': '../NETFLIX_DATA/partitions/validation/part_1_7.parquet', 'part': 1, 'group': 7}


In [15]:
# user and movie_map from full train data
user_map, movie_map = map_id(full_train_partitions)

Mapping IDs: 100%|██████████| 7/7 [00:01<00:00,  3.90it/s]

Map successful for 431670 users, 3720 movies


In [16]:
# preload user rating profiles

full_train_data = AutoEncoder.load_user_data(partitions=full_train_partitions, 
                                             user_map=user_map)

full_validation_data = AutoEncoder.load_validation_data(partitions=full_val_partitions, 
                                                        user_map=user_map,
                                                        movie_map=movie_map)

Loading validation pairs 7/7: 100%|██████████| 353005/353005 [00:12<00:00, 28047.50it/s]


In [18]:
final_save_dir = checkpoint_dir+"/final"

final_params = {
    'num_epochs': 30,
    'checkpoint_interval': 5,
    'eval_interval': float('inf'), # no eval
    'batch_size': 512
}

tuned_model, rmse = train_autoencoder(
    train_partitions=full_train_partitions,
    validation_partitions=full_val_partitions,
    user_map=user_map,
    movie_map=movie_map,
    checkpoint_dir=final_save_dir,
    user_data=full_train_data,
    validation_data=full_validation_data,
    **study.best_params,
    **final_params
)

Device: cuda


Calculating global mean: 100%|██████████| 7/7 [00:00<00:00,  7.38it/s]


Global mean rating: 3.607


Epoch 1/30: 100%|██████████| 844/844 [01:26<00:00,  9.76it/s, loss=1.0275]


Epoch 1 - Average Loss: 1.6219


...

Epoch 29/30: 100%|██████████| 844/844 [01:25<00:00,  9.84it/s, loss=0.8746]


Epoch 29 - Average Loss: 0.8743


Epoch 30/30: 100%|██████████| 844/844 [01:25<00:00,  9.89it/s, loss=0.8070]


Epoch 30 - Average Loss: 0.8751
Saved checkpoint at ../model_checkpoints/final/checkpoint_epoch_30.pth.gz
Saved final model at ../model_checkpoints/final/final_model.pth.gz


UnboundLocalError: cannot access local variable 'val_rmse' where it is not associated with a value

In [19]:
model_path = checkpoint_dir+"/final/final_model.pth.gz"

In [20]:
print(f"Loading model from {model_path}")

# Load the checkpoint
with gzip.open(model_path, 'rb') as f:
    checkpoint = torch.load(f, map_location=device)

# Extract metadata
metadata = checkpoint['metadata']
print(f"Model metadata: {metadata}")

# Initialize model with same architecture
model = AutoEncoder(
    num_movies=len(movie_map),
    hidden_dims=metadata['hidden_dims'],
    dropout=metadata['dropout'],
    l2=metadata.get('l2_reg', 0.001),  # Use l2_reg from metadata if available
    global_mean=checkpoint.get('global_mean', 3.0)  # Try to get from checkpoint
).to(device)

# Load the trained weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set to evaluation mode

print(f"Model loaded successfully!")
print(f"Architecture: {metadata['hidden_dims']}")
print(f"Dropout: {metadata['dropout']}")
print(f"Learning rate (training): {metadata['learning_rate']}")

Loading model from ../model_checkpoints/final/final_model.pth.gz
Model metadata: {'epochs': 30, 'train_loss': 0.8751115782707224, 'val_loss': None, 'hidden_dims': [1024, 256, 128], 'dropout': 0.4807743363021898, 'learning_rate': 0.00025806243863933984}
Model loaded successfully!
Architecture: [1024, 256, 128]
Dropout: 0.4807743363021898
Learning rate (training): 0.00025806243863933984


In [21]:
# prepare test data
test_path = "../NETFLIX_DATA/test.parquet"
test_df = pd.read_parquet(test_path)
test_data = [(row['user_id'], row['movie_id'], row['rating']) 
             for _, row in tqdm(test_df.iterrows(), 
                               total=len(test_df), 
                               desc="Loading test data")]

Loading test data: 100%|██████████| 1408395/1408395 [00:48<00:00, 28836.62it/s]


In [22]:
test_metrics = evaluate_model(
    model=model,
    validation_data=test_data,        
    training_data=full_train_data, 
    user_map=user_map,               
    movie_map=movie_map,             
    device=device
)

Evaluating: 100%|██████████| 2751/2751 [00:16<00:00, 167.11it/s]


In [23]:
print(f"Evaluated on test data of length {len(test_data)}")
test_metrics

Evaluated on test data of length 1408395


{'loss': 1.5933213663980612,
 'rmse': 1.2622683416762306,
 'mae': 1.0228561228930335,
 'total_predictions': 1408395}